# Assignment 6
This notebook contains code of implementation of area under curve calculation using trapezoidal rule.
It also has the code used for testing the performance and accuracy, and markdown used in places to highlight the reasoning behind the choices I have made.
All of the markdown text and important points from comments is in the report as well. So, I intend for you to only have to read my report, this is just if you wanted to check how I was testing.

In [1]:
%reload_ext Cython

In [2]:
# Imports
import numpy as np
import math as math

## Trapezoidal rule function implementations

In [3]:
# Pure python
def py_trapz(f, a, b, n):
    h = (b - a)/n
    area = 0.5*(f(a)+f(b))
    x=a+h
    for i in range(n-1):
        area += f(x)
        x+=h
    return area*h

In [4]:
%%cython --annotate
# So that we can see the C code generated corresponding to each line of code

import cython
@cython.cdivision(True) # Decorator to skip devide by zero error catching done in python division
cpdef double cy_trapz(f, double a, double b, int n):
    cdef double h = (b - a)/n
    cdef double area = 0.5*(f(a)+f(b))
    cdef double x=a+h
    for i in range(n-1):
        area += f(x)
        x+=h
    return area*h

In [5]:
# Function that calls np.trapz
def np_trapz(f, a, b, n):
    vf = np.vectorize(f) # Some functions may be callable for a scalar value only. For Eg: math.sin()
    outputs = vf(np.linspace(a, b, n+1))
    return np.trapz(outputs, dx=(b - a)/n)

In [6]:
def square(x):
    return x*x
def sine(x):
    return math.sin(x)
def exponential(x):
    return math.exp(x)
def reciprocal(x):
    return 1/x

In [7]:
# Testing
print("square")
print(py_trapz(square, 0, 1, 10000))
print(cy_trapz(square, 0, 1, 10000))
print(np_trapz(square, 0, 1, 10000))
%timeit py_trapz(square, 0, 1, 10000)
%timeit cy_trapz(square, 0, 1, 10000)
%timeit np_trapz(square, 0, 1, 10000)

print("sine")
print(py_trapz(sine, 0, np.pi, 10000))
print(cy_trapz(sine, 0, np.pi, 10000))
print(np_trapz(sine, 0, np.pi, 10000))
%timeit py_trapz(sine, 0, np.pi, 10000)
%timeit cy_trapz(sine, 0, np.pi, 10000)
%timeit np_trapz(sine, 0, np.pi, 10000)

print("exponential")
print(py_trapz(exponential, 0, 1, 10000))
print(cy_trapz(exponential, 0, 1, 10000))
print(np_trapz(exponential, 0, 1, 10000))
%timeit py_trapz(exponential, 0, 1, 10000)
%timeit cy_trapz(exponential, 0, 1, 10000)
%timeit np_trapz(exponential, 0, 1, 10000)

print("reciprocal")
print(py_trapz(reciprocal, 1, 2, 10000))
print(cy_trapz(reciprocal, 1, 2, 10000))
print(np_trapz(reciprocal, 1, 2, 10000))
%timeit py_trapz(reciprocal, 1, 2, 10000)
%timeit cy_trapz(reciprocal, 1, 2, 10000)
%timeit np_trapz(reciprocal, 1, 2, 10000)

square
0.33333333499994316
0.33333333499994316
0.33333333500000006
872 µs ± 9.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
710 µs ± 8.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.34 ms ± 22.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sine
1.9999999835503668
1.9999999835503668
1.999999983550659
1.34 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.26 ms ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.87 ms ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
exponential
1.718281829890868
1.718281829890868
1.7182818298909468
1.39 ms ± 30.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.17 ms ± 6.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.81 ms ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
reciprocal
0.6931471811849667
0.6931471811849667
0.6931471811849453
955 µs ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


As we can see, the cython optimizations helped to increase the speed slightly, and accuracy remained similar across implementations. 

Interestingly, numpy performed the worst. This is because numpy is optimized for applying functions over large number of inputs in arrays. By removing the np.vectorize line and directly applying numpy functions, we get far lower execution times.

This does make the other two implementations slower, as numpy functions are not optimized and slow when we apply them on scalar values, as we can see below.

In [8]:
# Redefining functions as their numpy equivalents
def square(x):
    return np.square(x)
def sine(x):
    return np.sin(x)
def exponential(x):
    return np.exp(x)
def reciprocal(x):
    return np.reciprocal(x)

# Function that calls np.trapz without linspace
def np_trapz(f, a, b, n):
    outputs = f(np.linspace(a, b, n+1))
    return np.trapz(outputs, dx=(b - a)/n)


In [9]:
# Testing

print("square")
print(py_trapz(square, 0, 1, 10000))
print(cy_trapz(square, 0, 1, 10000))
print(np_trapz(square, 0, 1, 10000))
%timeit py_trapz(square, 0, 1, 10000)
%timeit cy_trapz(square, 0, 1, 10000)
%timeit np_trapz(square, 0, 1, 10000)

print("sine")
print(py_trapz(sine, 0, np.pi, 10000))
print(cy_trapz(sine, 0, np.pi, 10000))
print(np_trapz(sine, 0, np.pi, 10000))
%timeit py_trapz(sine, 0, np.pi, 10000)
%timeit cy_trapz(sine, 0, np.pi, 10000)
%timeit np_trapz(sine, 0, np.pi, 10000)

print("exponential")
print(py_trapz(exponential, 0, 1, 10000))
print(cy_trapz(exponential, 0, 1, 10000))
print(np_trapz(exponential, 0, 1, 10000))
%timeit py_trapz(exponential, 0, 1, 10000)
%timeit cy_trapz(exponential, 0, 1, 10000)
%timeit np_trapz(exponential, 0, 1, 10000)

print("reciprocal")
print(py_trapz(reciprocal, 1, 2, 10000))
print(cy_trapz(reciprocal, 1, 2, 10000))
print(np_trapz(reciprocal, 1, 2, 10000))
%timeit py_trapz(reciprocal, 1, 2, 10000)
%timeit cy_trapz(reciprocal, 1, 2, 10000)
%timeit np_trapz(reciprocal, 1, 2, 10000)

square
0.33333333499994316
0.33333333499994316
0.33333333500000006
7.32 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.32 ms ± 47.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
46.3 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
sine
1.9999999835503668
1.9999999835503668
1.999999983550659
7.3 ms ± 95.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.39 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
159 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
exponential
1.718281829890868
1.718281829890868
1.7182818298909468
8.07 ms ± 301 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.65 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
116 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
reciprocal
0.6931221811849666
0.6931471811849667
0.6931471811849453
7.67 ms ± 179 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.61 ms ± 130 µs